<a href="https://colab.research.google.com/github/FyodorPi/dashboard/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install dash pandas plotly

In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import plotly.express as px
import openpyxl
import xlrd
import os
import glob
import statistics as stat
import math
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Color, PatternFill, Font, Border, Alignment
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import ColorScaleRule, CellIsRule, FormulaRule
from openpyxl.chart import BarChart, LineChart, ScatterChart, Reference, Series
from openpyxl.worksheet.table import Table, TableStyleInfo
from pandas.io.excel import ExcelWriter
import matplotlib.pyplot as plt
import statsmodels.api as sm
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# product = ('arm_kanat', 'Арматурный канат')
# product = ('prgon', 'Противоугон')
product = ('p_prov', 'Пружинная проволока')
# os.chdir('..')
os.chdir(f"/content/drive/My Drive/1Статистика/Статистика/oracle_{product[0]}")

xl_files = glob.glob('*')
combined = pd.DataFrame()
combined2 = pd.DataFrame()
combined3 = pd.DataFrame()

# Цикл по файлам
for xl_file in xl_files:
    xl_file_obj = pd.ExcelFile(xl_file)

    for sheet_name in xl_file_obj.sheet_names:
        results = 'Результаты'
        probes = 'Пробы'
        vhcontrol = 'Вх.контроль'

        if sheet_name.startswith(results):
            data = pd.read_excel(xl_file_obj,
                                # skiprows=15,
                                header=[0],
                                sheet_name=sheet_name)

            data = data.drop(labels = [0],axis = 0)
            data=data.rename(columns = {'Unnamed: 10':'Минимум_норм', 'Unnamed: 11':'Максимум_норм'})
            # print(data)
            data = data.astype({'Минимум': np.float64})
            data = data.astype({'Максимум': np.float64})
            data = data.dropna (subset=['Минимум'])
            data = data.dropna (subset=['Максимум'])
            data['mean'] = data[['Минимум', 'Максимум']].mean(axis= 1 )

            combined = pd.concat([combined, data], sort=False)
            # print(combined)

        if sheet_name.startswith(probes):
            data2 = pd.read_excel(xl_file_obj,
                                header=[0],
                                sheet_name=sheet_name)
            combined2 = pd.concat([combined2, data2], sort=False)
            combined2["Диаметр"] = combined2["Описание позиции"].str.extract('(\d+(?:,\d+)?\s)', expand=False)
            combined2['Диаметр'] = combined2['Диаметр'].str.replace(',', '.')
            # combined2['Диаметр'] = combined2['Диаметр'].str.replace(' ', '')
            combined2 = combined2.astype({'Диаметр': np.float64})
            combined2['Размер-марка-НД'] = combined2['Описание позиции'].str.replace('Проволока ', '')

        if sheet_name.startswith(vhcontrol):
            data3 = pd.read_excel(xl_file_obj,
                                header=[0],
                                sheet_name=sheet_name)
            data3 = data3.drop(labels = [0],axis = 0)
            combined3 = pd.concat([combined3, data3], sort=False)
            # print(combined3)

# elem = 'C'
elem = 'Mn'

combined4 = combined3.loc[combined3["Показатель"] == elem]
# combined5 = combined3.loc[combined3["Показатель"] == 'Врм_сопр']
combined4 = combined4.astype({'Данные СК поставщика': np.float64})
# combined5 = combined5.astype({'Данные СК поставщика': np.float64})
combined4 = combined4.groupby(['Плавка'])['Данные СК поставщика'].agg(["mean"])
combined4 = combined4.rename(columns={'mean': elem})
# combined4 = combined4.rename(columns={'mean': 'Ср. Mn'})
# combined5 = combined5.groupby(['Плавка'])['Данные СК поставщика'].agg(["mean"])
# combined5 = combined5.rename(columns={'mean': 'Ср. Врм_сопр'})

combine=pd.merge(combined, combined2, on=('Проба'), how ='left')
combine=pd.merge(combine, combined4, on=('Плавка'), how ='left')
# combine=pd.merge(combine, combined5, on=('Плавка'), how ='left')
# print(combine)

combine = pd.DataFrame(data=combine)

combine['Код проверки'] = combine['Код проверки'].str.replace('|', '-')
combine['Код проверки'] = combine['Код проверки'].str.replace('/', '-')
combine['Код проверки'] = combine['Код проверки'].str.replace('-М', '')
combine['Класс проверки (код)'] = combine['Класс проверки (код)'].str.replace('|', '-')
combine['Класс проверки (код)'] = combine['Класс проверки (код)'].str.replace('М-', '')

# Удаление строк с пустыми значениями в поле "Минимум"
combine = combine.dropna (subset=['Минимум'])

print(combine.columns)
# print(combine)

<>:44: SyntaxWarning:

invalid escape sequence '\d'

<>:44: SyntaxWarning:

invalid escape sequence '\d'

/tmp/ipython-input-1664392487.py:44: SyntaxWarning:

invalid escape sequence '\d'



Index(['Проба', 'Партия_x', 'Класс проверки (код)', 'Код проверки',
       'Контрольное значение', 'Минимум', 'Максимум', 'Е/И',
       'Факультативность', 'нормативные значения', 'Минимум_норм',
       'Максимум_норм', 'Дата ввода результатов', 'Позиция',
       'Описание позиции_x', 'Примечание контролера_x', 'Статус пробы_x',
       'mean', 'Код цеха', 'Участок', 'Код позиции', 'Описание позиции_y',
       'Партия_y', 'Код марки стали', 'Описание марки', 'Плавка',
       'Дата производства', 'Статус пробы_y', 'Дата создания пробы',
       'Примечание контролера_y', 'Примечание лаборанта',
       'Заказ, строка пробы', 'Потребитель (проба)',
       'Номер стандарта на тех.требования 1',
       'Наименование стандарта на техтребования 1',
       'Номер стандарта на сортамент (марку)',
       'Наименование стандарта на сортамент (марку)', 'Диаметр',
       'Размер-марка-НД', 'Mn'],
      dtype='object')


In [9]:
for klass in ['МЕХСВ']:
# for klass in combine['Класс проверки (код)'].unique ():
    print(klass)
    # Выборка по классу свойств
    combin = combine.loc[combine["Класс проверки (код)"] == klass]

    # for charact in combin['Код проверки'].unique ():
    for charact in [
        'Врм_сопр',
        # 'Число_скручиваний',
        ]:
        # Выборка по свойству
        combi = combin.loc[combin["Код проверки"] == charact]

        # for d_marka_nd in combi['Размер-марка-НД'].unique ():
        for d_marka_nd in [
            # 'Б-2-1,7 ГОСТ 9389-75 (СПИК)',
            # 'Б-2-1,75 ГОСТ 9389-75 (СПИК)',
            'Б-2-1,8 ГОСТ 9389-75 (СПИК)',
            ]:
            # print(marka)
            # Выборка по марке стали
            comb = combi.loc[combi["Размер-марка-НД"] == d_marka_nd]
            # comb = combi.loc[combi["Марка-НД"] == 'Б-2-1,8 ГОСТ 9389-75 (СПИК)']

            # for diam in comb['Диаметр'].unique ():
            #     # print(diam)
            #     # Выборка по состоянию
                # com = comb.loc[comb["Диаметр"] == diam]
            for minimum in comb['Минимум_норм'].unique ():
                    # print(min)
                # Выборка по нижней технической границе
                    df1 = comb.loc[comb['Минимум_норм'] == minimum]
                    # print(df['Максимум'].unique ())
                    for maximum in df1['Максимум_норм'].unique ():
                        # print(max)
                # Выборка по верхней технической границе
                        df = df1.loc[df1['Максимум_норм'] == maximum]
                        pd.options.mode.chained_assignment = None
                        df['Дата создания пробы1'] = df['Дата создания пробы']
                        df['Дата создания пробы1'] = pd.to_datetime(df["Дата создания пробы1"],
                                                                        format="%d.%m.%Y %H:%M:%S")
                        df = df.sort_values('Дата создания пробы1')
                        df['Дата создания пробы1'] = pd.to_datetime(df['Дата создания пробы1']).dt.date
                        df['Дата создания пробы1']=pd.to_datetime(df['Дата создания пробы1'].astype(str))
                        df['Дата создания пробы1'] = df['Дата создания пробы1'].dt.strftime('%d.%m.%Y')
                        # pd.options.display.max_columns = None
                        df = df.loc[df["Описание марки"] == '65']
                        # print(df["Описание марки"])

                        dates = df['Дата создания пробы1'].tolist()
                        strength_values = df['mean'].tolist()
                        batches = df['Плавка'].tolist()

                        # Генерация случайных данных
                        # np.random.seed(42)
                        # data_size = 100
                        # dates = pd.date_range(start="2023-01-01", periods=data_size, freq='D')
                        # batches = [f'Партия {i+1}' for i in range(10)]
                        data = {
                            'Плавка': batches,
                            'Дата производства': dates,
                            'Значение': strength_values
                        }

                        df = pd.DataFrame(data)

                        # Создание Dash приложения
                        app = dash.Dash(__name__)

                        app.layout = html.Div([
                            dcc.Dropdown(
                                id='batch-dropdown',
                                options=[{'label': batch, 'value': batch} for batch in df['Плавка'].unique()],
                                multi=True,
                                placeholder='Выберите партии'
                            ),
                            dcc.DatePickerRange(
                                id='date-picker',
                                start_date=df['Дата производства'].min(),
                                end_date=df['Дата производства'].max(),
                                display_format='YYYY-MM-DD'
                            ),
                            dcc.Graph(id='value-graph')
                        ])

                        @app.callback(
                            Output('value-graph', 'figure'),
                            [Input('batch-dropdown', 'value'),
                            Input('date-picker', 'start_date'),
                            Input('date-picker', 'end_date')]
                        )
                        def update_graph(selected_batches, start_date, end_date):
                            if selected_batches is None:
                                filtered_df = df
                            else:
                                filtered_df = df[(df['Плавка'].isin(selected_batches)) &
                                                (df['Дата производства'] >= start_date) &
                                                (df['Дата производства'] <= end_date)]

                            fig = px.scatter(filtered_df, x='Дата производства', y='Значение', color='Плавка',
                                            title='Значения временного сопротивления ' + d_marka_nd + ' по дате и плавке',
                                            labels={'Значение': 'Временное сопротивление, Н/мм2', 'Дата производства': 'Дата'})

                            return fig

                        if __name__ == '__main__':
                            # app.run_server(debug=True)
                            app.run(debug=True)


МЕХСВ


<IPython.core.display.Javascript object>